In [8]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
import xgboost as xgb
import matplotlib
import torch
import torch.utils.data as data_utils
import torch.nn as nn
import tensorflow as tf
import imblearn
from sklearn.model_selection import train_test_split
import collections
from sklearn.metrics import roc_curve, auc
import pymoo
import random

In [3]:
data_dir='/grid_mnt/vol_home/llr/cms/hakimi/HGCAL/New_vars' # output from alternative_var.ipynb
# select signal and background (not improtant for this estiamtion, I used elec vs PU)
signal='elec' 
bkg = 'PU' 
df_sig= pd.read_csv(data_dir+'/{}_df.csv'.format(signal), low_memory=True)
df_bkg=pd.read_csv(data_dir+'/{}_df.csv'.format(bkg), low_memory=True)
#join dfs
data=df_sig.append(df_bkg)
savedir =os.getcwd() + '/moo/'+ bkg + '/'
os.makedirs(savedir, exist_ok=True)

import matplotlib
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

# Config

In [4]:
commonfeat=True #use set of variable optimal vs PU and pions at the same time, else use optimal for each case

if commonfeat==False:

    if bkg == 'pions':
        features =['abseta', 'cl3d_firstlayer', 'varee', 'varpp', 'varzz', 'varrr',
                     'cl3d_meanz', 'cl3d_showerlength','cl3d_coreshowerlength','EoT',
                    'Emax_5', 'ebm1', 'Emax_4R', 'hbm', 'firstHcal_1', 'reverse_ebm1']


    elif bkg == 'PU':
        features = ['abseta', 'cl3d_firstlayer', 'varee', 'varpp', 'varzz', 'varrr',
                     'cl3d_meanz', 'cl3d_showerlength','cl3d_coreshowerlength', 'EoT',
                    'firstHcal_5', 'Emax_4R', 'ebm1', 'Emax_2R', 'firstHcal_1', 'Emax_5']

    elif bkg == 'QCD':
        features=  ['abseta', 'cl3d_firstlayer', 'varee', 'varpp', 'varzz', 'varrr',
                     'cl3d_meanz', 'cl3d_showerlength','cl3d_coreshowerlength', 'EoT','ebm1', 'Emax_4R',
                    'reverse_ebm1', 'first_2', 'Emax_2R', 'Emax_5', 
                    ]
    elif bkg == 'PU+QCD':
        features = ['abseta', 'cl3d_firstlayer', 'varee', 'varpp', 'varzz', 'varrr',
                     'cl3d_meanz', 'cl3d_showerlength','cl3d_coreshowerlength', 'EoT',
                    'firstHcal_5', 'Emax_4R', 'ebm1', 'Emax_2R', 'firstHcal_1', 'Emax_5']
elif commonfeat==True:
    features = ['abseta', 'cl3d_firstlayer', 'varee', 'varpp', 'varzz', 'varrr',
                     'cl3d_meanz', 'cl3d_showerlength','cl3d_coreshowerlength', 'EoT',
                    'firstHcal_5', 'Emax_4R', 'ebm1', 'Emax_2R', 'firstHcal_1', 'Emax_5', 'reverse_ebm1', 'hbm']

In [6]:
#chose signal and background
#balancing initialization

data['signal']= 5
data['weight']=0
data['signal'][data['sample']==signal]= 1
data['signal'][data['sample']!=signal] = 0
balancing='comb' #'SMOTE', 'weights', 'ada', 'under', 'comb'

/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


# Multi objective optimization

## 1. Select genes
The genes are the different paramaters of the optimization

-> alleles for each gene is the number of bits [2-16]

-> add exception for discrete variables: 2**nbits <= range

In [69]:
def maxbits(feat, maxbit):
    if feat.dtype == 'float64':
        return maxbit
    elif feat.dtype =='int64':
        nval = len(np.unique(feat))
        print(nval)
        for i in range(1,maxbit) : 
            if 2**i >= nval: return(i)
        return(i)

#change discrete functuions type to int
discrete_cols = ['cl3d_showerlength','cl3d_coreshowerlength', 'cl3d_firstlayer', 'cl3d_maxlayer' ]
data[discrete_cols]=data[discrete_cols].astype('int64')


In [7]:
#preprocessing and balancing
X_train, X_test, y_train, y_test = train_test_split(data[features], 
                                                    data[['signal','weight','cl3d_eta','cl3d_pt', 'genpart_pt', 'genpart_exeta', 'sample']],
                                                    test_size=0.2,random_state=42)
train_untouched=xgb.DMatrix(data=X_train,label=y_train['signal'],feature_names=features)
y_train_untouched=y_train
X_train_untouched=X_train
over_ratio = 0.4 #0.2 
under_ratio = 0.5
if bkg == 'QCD':
    over_ratio = 0.8
    under_ratio = 1
over=imblearn.over_sampling.SMOTE(sampling_strategy=over_ratio, random_state=42)
under= imblearn.under_sampling.RandomUnderSampler(sampling_strategy=under_ratio, random_state=42)
X_train['cl3d_pt']=y_train['cl3d_pt']
X_train['cl3d_eta']=y_train['cl3d_eta']
X_train, y_train=over.fit_resample(X_train,y_train['signal'])
X_train, y_train=under.fit_resample(X_train, y_train)
y_train=pd.DataFrame(y_train)
y_train['weight']=1
y_train['cl3d_pt']=X_train['cl3d_pt']
y_train['cl3d_eta']=X_train['cl3d_eta']
X_train.drop('cl3d_pt', axis=1, inplace=True)
X_train.drop('cl3d_eta', axis=1, inplace=True)


#dump files for easy restart
datadir='/grid_mnt/vol_home/llr/cms/hakimi/HGCAL/moo/data/'
if not os.path.exists(datadir):
        os.makedirs(datadir)
X_test.to_csv(datadir+'X_test_{}.csv'.format(bkg))
y_test.to_csv(datadir+'y_test_{}.csv'.format(bkg))
X_train.to_csv(datadir+'X_train_{}.csv'.format(bkg))
y_train.to_csv(datadir+'y_train_{}.csv'.format(bkg))

KeyboardInterrupt: 

In [79]:
#MOO problem definition

#MOO inputs: nvar [0-16], max boost rounds[10-500], max_depth[2-5], eta [1-5] (negative power of 10)

#number of optimization parameters
nvar = len (data[features].columns) + 3 #1 for each feature + max boost rounds, max depth and learning rate
print('nvar',nvar)

#number of objetives
nobj = 3 # efficiency and number of bits used + model complexity

#number of constraints
nconstr = 0

bitmin= 0 #0 if we want to remove features

# variables lower bound xl
#features number of bits
xl=[bitmin for feat in data[features].columns]
#add minimum for bdt params
xl.extend([10,2,1])
print('lower bounds',xl)


bitmax = 16
#upper bound
#features number of bits
xu =[maxbits(data[feat], bitmax) for feat in data[features].columns]
#add bdt params
xu.extend([500,5,5])
print ('upper bounds',xu)


#BDT params
param = {
        'nthread' : 8,
        # Parameters that we are going to tune.
        'max_depth':2,
        'eta':0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        # Other parameters
        'objective':'binary:logistic',
        'eval_metric': 'auc', 
        'alpha':0.01,
        'lambda':10,
    }

nvar 21
lower bounds [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1]
20
49
33
1955
1955
47278
upper bounds [16, 5, 16, 16, 16, 16, 16, 6, 6, 16, 16, 16, 11, 16, 16, 16, 11, 15, 500, 5, 5]


In [81]:
# quantization
def quantize(feat, nbits, method, fmin, fmax): #feat is the feature to quantize, nbist the number of bits, method: 'uniform' or 'percentile'
    nbins= 2**nbits
    if feat.dtype == 'float64':
        method = method
    elif feat.dtype == 'int64':
        method = 'uniform'
    if method == 'uniform':
        bins=np.linspace(fmin, fmax, nbins)
    elif method == 'percentile':
        bins=[np.percentile(feat, n) for n in np.linspace(0,100,nbins+1)]
    else :
        print('Error: Invalid method')
        return
    
    
    #print(feat.dtype,nbits, method)
    return np.digitize(feat, bins, right=True)


def quantize_ebm0(x,nb):
    qbm=[]
    nlay=14
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.ebm0)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,14,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm


def quantize_ebm1(x,nb):
    qbm=[]
    nlay=14
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.ebm1)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,14,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm
def quantize_hbm(x,nb):
    qbm=[]
    nlay=21
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.hbm)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,nlay,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm
def quantize_reverse_ebm0(x,nb):
    qbm=[]
    nlay=14
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.reverese_ebm0)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,14,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm
def quantize_reverse_ebm1(x,nb):
    qbm=[]
    nlay=14
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm =int(x.reverese_ebm1)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,14,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm
def quantize_reverese_hbm(x,nb):
    qbm=[]
    nlay=21
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.reverse_hbm)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,nlay,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm

In [86]:
#training function
def train_quantized(x):
    #print('Parameters = {}'.format(x))
    nfeat=len (data[features].columns)
    param['eta']= float(10)**((-1)*x[nfeat+2])
    param['max_depth']=x[nfeat+1]

    num_boost_round=x[nfeat]
    #print('test : boost rounds {} max_depth {} eta {}'.format(num_boost_round, param['max_depth'], param['eta']))
    
    qtrain=pd.DataFrame()
    qtest=pd.DataFrame()
    feat_kept=features.copy()
    tot_bits= 0
    for i,feat in enumerate(X_train.columns):
        #print("feature :", feat, "quantized on {} bits".format(x[i]))
        if x[i]==0: #drop feat
            #print("dropping {} from {}".format(feat, feat_kept))
            feat_kept.remove(feat)
        elif 'bm' not in 'feat':
            fmin, fmax= X_train[feat].min(), X_train[feat].max(), 
            qtrain[feat]=quantize(X_train[feat], x[i], 'uniform', fmin, fmax)
            qtest[feat]=quantize(X_test[feat], x[i], 'uniform', fmin, fmax)
            tot_bits += x[i]
        elif feat == 'ebm0':
            qtrain[feat]=X_train.apply(quantize_ebm0, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_ebm0, nb = x[i], axis=1)
            nlay=14
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
        elif feat == 'ebm1':
            qtrain[feat]=X_train.apply(quantize_ebm1, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_ebm1, nb = x[i], axis=1)
            nlay=14
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
        elif feat == 'hbm':
            qtrain[feat]=X_train.apply(quantize_hbm, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_hbm, nb = x[i], axis=1)
            nlay=21
            bits=nlay/int(np.ceil(nlay/x[i]))
            tot_bits += nlay/int(np.ceil(nlay/x[i]))
        elif feat == 'reverse_ebm0':
            qtrain[feat]=X_train.apply(quantize_reverse_ebm0, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_reverse_ebm0, nb = x[i], axis=1)
            nlay=14
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
        elif feat == 'reverse_ebm1':
            qtrain[feat]=X_train.apply(quantize_reverse_ebm1, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_reverse_ebm1, nb = x[i], axis=1)
            nlay=14
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
        elif feat == 'reverse_hbm':
            qtrain[feat]=X_train.apply(quantize_reverse_hbm, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_reverse_hbm, nb = x[i], axis=1)
            nlay=21
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
    train= xgb.DMatrix(data=qtrain,label=y_train['signal'], feature_names=feat_kept, weight=y_train['weight'])
    test= xgb.DMatrix(data=qtest,label=y_test['signal'],feature_names=feat_kept)
    #use gpu if available
    gpu=tf.test.is_gpu_available(
        cuda_only=False, min_cuda_compute_capability=None
    )
    if gpu==True:
        param['tree_method']='gpu_hist'

    #Now train and fit best parameters BDT
    eval_result = {}
    BDT = xgb.train(
        param,
        train,
        num_boost_round=num_boost_round,
        #feval = SoverB,
        evals=[(train, 'Train'),(test, "Test")],
        evals_result = eval_result,
        verbose_eval=None,
    )
    pred= BDT.predict(test)


    fpr, tpr, threshold = roc_curve(y_test['signal'],pred, pos_label=1)
    fpr.sort()
    tpr.sort()
    roc_test=pd.DataFrame({'tpr':tpr,'fpr':fpr, 'threshold':threshold})

    #return auc at > cut sig eff
    cut=0.8

    roc_thr=roc_test[roc_test['tpr']>0.8]
    if len(roc_thr)>3:
        f1= (-1)*auc(roc_thr['fpr'], roc_thr['tpr']) # negative to be a minimization
    else : 
        f1 = 0
    #f2 = total number of bits used
    f2=tot_bits
    
    #f3: compute estimation of bdt ressource usage based on LUT_estimation table
    f3=0
    for i in BDT.trees_to_dataframe()[BDT.trees_to_dataframe()['Feature']!='Leaf'].iterrows():
        depth=min(5,param['max_depth'])
        feat=i[1]['Feature']
        feat_prec= x[X_train.columns.get_loc(feat)]
        if feat_prec < 3: feat_prec =3 #estimation is not reliable under 3 bits precision, use conservative estimate value
        #print('feature : {}, prec : {} , depth: {}'.format(feat, feat_prec,param['max_depth'] ))
        price=proxy.query('precision == {} & depth =={} '.format(feat_prec, depth))['coef'].values[0] 
        f3 += price
    
    f3 = round(f3)
    #print('bkg efficiency = {}, using {} bits and {} LUT'.format(f1, f2, f3))
    return ([f1, f2, f3], pred, BDT, qtrain, feat_kept)

In [88]:
#plot shap values of model
def plot_shap(BDT, data, feats):
    import shap
    # select a set of background examples to take an expectation over
    #background = X_train.to_numpy()[np.random.choice(X_train.shape[0], 1000, replace=False)]

    # explain predictions of the model on four images
    explainer = shap.Explainer(BDT)
    a= qtrain[feat_kept][(y_train['signal']==1).values].sample(2000)
    b = qtrain[feat_kept][(y_train['signal']==0).values].sample(2000)
    shap_df= pd.concat([a,b])

    #shap_values = explainer.shap_values(X_test.to_numpy()[np.random.choice(X_test.shape[0], 1000, replace=False)])
    shap_values = explainer.shap_values(shap_df.to_numpy())
    col_names=[val for val in shap_df.columns if val != 'abseta']
    col_names.append('abseta')

    plt.figure(figsize=(10,5))
    #shap.summary_plot(shap_values, X_test.to_numpy()[np.random.choice(X_test.shape[0], 1000, replace=False)], 
    #                  features,max_display=99, show=False, alpha = 0.7 )

    shap.summary_plot(shap_values, shap_df.to_numpy() ,shap_df.columns,max_display=99, show=False, alpha = 0.8 )
    plt.xlabel('Shap value', fontsize=16)

# Run MOO

In [89]:
import numpy as np

from pymoo.factory import get_algorithm, get_crossover, get_mutation, get_sampling, get_selection
from pymoo.optimize import minimize
from pymoo.model.problem import Problem


class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=nvar, #number of variables in the problem (ie hyperparameters)
                         n_obj=nobj, #number of objectives
                         n_constr=nconstr, #number of constraints
                         xl=xl, #lower bound for variables
                         xu=xu, #higher bound for variables
                         type_var=int, 
                         elementwise_evaluation=True) #Only one solution is evaluated at a time, 
                                                      #vectorized or functional evaluation supported 

    def _evaluate(self, x, out, *args, **kwargs):        
        out["F"] = train_quantized(x)[0] #black box function to evaluate, should return n_obj values
problem=MyProblem()

In [90]:
#initialization




def initial_pop(pop_size=16, rand= True):
    #initializatio
    #x[0-15] = nbits, x[16]=boost rounds, x[17]=max depth, x[18]=eta
    nfeat=len (data[features].columns)
    max_val=16
    pop_size=pop_size
    sampling=[]
    #hyperdiag
    for i in range(1,max_val+1):
        current=[i for j in range(nfeat)]
        #BDT params: min + i*range/12
        current.append(round(2+(i-1)*(498/(max_val-1)))) # boost rounds
        current.append(round(2+(i-1)*(3/(max_val-1)))) # depth
        current.append(round(1+(i-1)*(4/(max_val-1)))) # eta
        sampling.append(current)
    #add random pop until pop_size is reached
    if rand:
        for i in range(pop_size-max_val):
            current=[random.randint(0,max_val) for j in range(nfeat)]
            current.append(random.randint(2,500))
            current.append(random.randint(2,5))
            current.append(random.randint(1,5))
            sampling.append(current)
        sampling=np.array(sampling)
    return(sampling)

In [139]:
pd.options.mode.chained_assignment = None  # default='warn'    
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=DeprecationWarning)


#hyperdiagonal sampling:
max_val=16 #maximum precision
pop_size=100 #number of indivuiduals in first generation
sampling=initial_pop(pop_size, True)

proxy=pd.read_csv('conifer_table.csv') # output from LUT_estimation.ipynb

method = get_algorithm("nsga2", # Non-doiminated sorting algorithm II, may use nsga3 for more than 3 objectives
                       pop_size=pop_size, #number of individual at each generation
                       sampling=sampling, #initial population
                       crossover=get_crossover("int_sbx", prob=1.0, eta=3.0), #crossover function
                       mutation=get_mutation("int_pm", eta=3.0), # def 3 #mutation functin
                       eliminate_duplicates=True,
                       )


res = minimize(problem,
               method,
               termination=('n_gen', 40), #end point, 40 generation seems to be enough
               seed=42,
               save_history=True,
               verbose=True
               )

print("Best solution found: %s" % res.X)
print("Function value: %s" % res.F)
print("Constraint violation: %s" % res.CV)

n_gen |  n_eval |  n_nds  |     eps      |  indicator  
    1 |     100 |      18 |            - |            -


/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:40: RuntimeWarning: divide by zero encountered in power
  alpha = 2.0 - np.power(beta, -(self.eta + 1.0))
/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:45: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (self.eta + 1.0)))[mask]


    2 |     200 |      23 |  0.103174603 |        nadir


/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:40: RuntimeWarning: divide by zero encountered in power
  alpha = 2.0 - np.power(beta, -(self.eta + 1.0))
/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:45: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (self.eta + 1.0)))[mask]


    3 |     300 |      35 |  0.089088454 |        nadir


...

   38 |    3800 |     100 |  0.003078005 |            f


/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:40: RuntimeWarning: divide by zero encountered in power
  alpha = 2.0 - np.power(beta, -(self.eta + 1.0))
/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:45: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (self.eta + 1.0)))[mask]


   39 |    3900 |     100 |  0.003673980 |            f


/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:40: RuntimeWarning: divide by zero encountered in power
  alpha = 2.0 - np.power(beta, -(self.eta + 1.0))
/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:45: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (self.eta + 1.0)))[mask]


   40 |    4000 |     100 |  0.026785714 |        ideal
Best solution found: [[ 0  0  1 ... 77  2  4]
 [11  5  2 ... 71  2  1]
 [ 0  0  0 ...  2  2  4]
 ...
 [ 1  1  3 ...  7  3  1]
 [ 1  0  3 ...  7  3  1]
 [ 0  0  1 ...  4  3  1]]
Function value: [[-3.22444697e-01  6.00000000e+00  1.00000000e+01]
 [-9.97519855e-01  1.18000000e+02  9.19000000e+02]
 [-4.72701798e-01  1.20000000e+01  0.00000000e+00]
 [-9.98480350e-01  1.09000000e+02  4.82710000e+04]
 [-9.98479500e-01  1.01000000e+02  2.98330000e+04]
 [-6.84941060e-01  9.00000000e+00  1.20000000e+01]
 [-7.28307543e-01  1.20000000e+01  6.00000000e+00]
 [-8.79131295e-01  1.30000000e+01  0.00000000e+00]
 [-3.73321171e-01  9.00000000e+00  1.10000000e+01]
 [-9.98450638e-01  9.00000000e+01  2.42200000e+04]
 [-9.98382703e-01  6.80000000e+01  1.21150000e+04]
 [-9.41791641e-01  1.40000000e+01  0.00000000e+00]
 [-3.24164916e-01  9.00000000e+00  2.00000000e+00]
 [-8.85938370e-01  1.30000000e+01  9.00000000e+00]
 [-9.98478236e-01  9.60000000e+01  2.

In [140]:
moo_name= '40gen_diag_auc_mut10_3d_conifer_v2'

import pickle as pkl
pkl.dump(res, open(savedir + '{}.pkl'.format(moo_name), 'wb'))